## More complex SQL Queries: DQ Project

In [26]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [3]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


So there we have the list of all the tables in the database. In the cell below, I will be performing some simple SQL query to get familiar with the data.

### Customers and the Total Amount Spent

In [4]:
%%sql     
SELECT c.first_name || ' ' || c.last_name name, SUM(i.total) total
    FROM customer c
    INNER JOIN invoice i ON i.customer_id = c.customer_id 
    GROUP BY 1
    ORDER BY 1
    LIMIT 10;

Done.


name,total
Aaron Mitchell,70.28999999999999
Alexandre Rocha,69.3
Astrid Gruber,69.3
Bjørn Hansen,72.27000000000001
Camille Bernard,79.2
Daan Peeters,60.38999999999999
Dan Miller,95.03999999999999
Diego Gutiérrez,39.6
Dominique Lefebvre,72.27
Eduardo Martins,60.39


### Genre that Sells most in the USA

In [54]:
%%sql
WITH short AS
( SELECT g.name, SUM(il.quantity) AS quantity
    FROM genre g 
    INNER JOIN track t ON g.genre_id = t.genre_id
    INNER JOIN invoice_line il ON t.track_id = il.track_id
    INNER JOIN invoice i ON i.invoice_id = il.invoice_id
    INNER JOIN customer c ON c.customer_id = i.customer_id
    WHERE c.country = 'USA'
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 10 )

SELECT name, quantity
    FROM short

Done.


name,quantity
Rock,561
Alternative & Punk,130
Metal,124
R&B/Soul,53
Blues,36
Alternative,35
Latin,22
Pop,22
Hip Hop/Rap,20
Jazz,14


Based on the top three genre that are also among the new four genre we received for the USA, I would recommend we list:
* Hip-Hop
*  Pop
* Blues

In [63]:
%%sql
SELECT e.first_name || ' ' || e.last_name AS employee_name, SUM(i.total) AS total, e.hire_date
    FROM employee e
    LEFT JOIN customer c ON c.support_rep_id = e.employee_id
    INNER JOIN invoice i ON i.customer_id = c.customer_id
    GROUP BY 1


Done.


employee_name,total,hire_date
Jane Peacock,1731.510000000004,2017-04-01 00:00:00
Margaret Park,1584.0000000000032,2017-05-03 00:00:00
Steve Johnson,1393.9200000000028,2017-10-17 00:00:00


Undoubtedly, Jane is the top performer, however, we need to consider the fact that she was hired beofre Margaret and Steve. The same way Margaret was hired before Steve. This my justify Jane and Margaret having better sales than Steve

### Analyzing Sales by Country

In [25]:
%%sql
SELECT c.country, COUNT(c.customer_id) customer_no, SUM(i.total) total_sales
    FROM customer c
    INNER JOIN invoice i ON i.customer_id = c.customer_id
    GROUP BY 1
    ORDER BY 3 DESC
    LIMIT 10;

Done.


country,customer_no,total_sales
USA,131,1040.4899999999998
Canada,76,535.5900000000001
Brazil,61,427.68000000000006
France,50,389.0699999999999
Germany,41,334.62
Czech Republic,30,273.24000000000007
United Kingdom,28,245.52
Portugal,29,185.13000000000002
India,21,183.14999999999998
Ireland,13,114.83999999999997


### Albums vs Individual Tracks

In [27]:

%%sql

WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    )

SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    CAST(count(invoice_id) AS FLOAT) / (
                                         SELECT COUNT(*) FROM invoice
                                      ) percent
FROM
    (
    SELECT
        ifs.*,
        CASE
            WHEN
                 (
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_first_track ifs
    )
GROUP BY album_purchase;

Done.


album_purchase,number_of_invoices,percent
no,500,0.8143322475570033
yes,114,0.18566775244299674


Based on the data above, album purchase is approximately one-fifth of the total sales and so there is a high potential opportunity cost forgone if the company decides to st